In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
.appName('Caching in Spark Tables').enableHiveSupport().getOrCreate()

25/06/20 06:49:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [7]:
customer_schema = 'customer_id INT , name STRING , city STRING ,state STRING ,country STRING ,registartaion_date STRING , is_active BOOLEAN'

In [5]:
!hadoop fs -ls /data/

Found 6 items
-rw-r--r--   2 root               hadoop       5488 2025-06-02 15:26 /data/customers_100.csv
-rw-r--r--   2 geurecruitanandraj hadoop   10528211 2025-06-17 15:58 /data/customers_10mb.csv
-rw-r--r--   2 geurecruitanandraj hadoop  168541068 2025-06-17 15:56 /data/customers_150mb.csv
-rw-r--r--   2 geurecruitanandraj hadoop  343317147 2025-06-20 06:58 /data/customers_300mb.csv
-rw-r--r--   2 geurecruitanandraj hadoop    8628599 2025-06-17 15:58 /data/orders_10.csv
-rw-r--r--   2 geurecruitanandraj hadoop    7896035 2025-06-17 15:59 /data/shippings_10.csv


In [8]:
cust_df = spark.read.schema(customer_schema).csv('/data/customers_300mb.csv')

In [9]:
cust_df.write.format('csv').saveAsTable('default.customers_500mb')

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
25/06/20 07:03:43 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider csv. Persisting data source table `spark_catalog`.`default`.`customers_500mb` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
25/06/20 07:03:43 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


In [10]:
spark.sql('DESCRIBE default.customers_500mb').show()

+------------------+---------+-------+
|          col_name|data_type|comment|
+------------------+---------+-------+
|       customer_id|      int|   NULL|
|              name|   string|   NULL|
|              city|   string|   NULL|
|             state|   string|   NULL|
|           country|   string|   NULL|
|registartaion_date|   string|   NULL|
|         is_active|  boolean|   NULL|
+------------------+---------+-------+



In [11]:
spark.sql('DESCRIBE EXTENDED default.customers_500mb').show(truncate=False)

+----------------------------+---------------------------------------------------------+-------+
|col_name                    |data_type                                                |comment|
+----------------------------+---------------------------------------------------------+-------+
|customer_id                 |int                                                      |NULL   |
|name                        |string                                                   |NULL   |
|city                        |string                                                   |NULL   |
|state                       |string                                                   |NULL   |
|country                     |string                                                   |NULL   |
|registartaion_date          |string                                                   |NULL   |
|is_active                   |boolean                                                  |NULL   |
|                            |

In [13]:
!hadoop fs -ls -h /user/hive/warehouse/customers_500mb

Found 4 items
-rw-r--r--   2 root hadoop          0 2025-06-20 07:03 /user/hive/warehouse/customers_500mb/_SUCCESS
-rw-r--r--   2 root hadoop    128.0 M 2025-06-20 07:03 /user/hive/warehouse/customers_500mb/part-00000-dbbbbe46-7ea3-4c44-a12d-b154efc74e96-c000.csv
-rw-r--r--   2 root hadoop    128.0 M 2025-06-20 07:03 /user/hive/warehouse/customers_500mb/part-00001-dbbbbe46-7ea3-4c44-a12d-b154efc74e96-c000.csv
-rw-r--r--   2 root hadoop     71.4 M 2025-06-20 07:03 /user/hive/warehouse/customers_500mb/part-00002-dbbbbe46-7ea3-4c44-a12d-b154efc74e96-c000.csv


In [15]:
spark.sql('SELECT * FROM customers_500mb limit 5').show()

+-----------+----------+------+-----------+-------+------------------+---------+
|customer_id|      name|  city|      state|country|registartaion_date|is_active|
+-----------+----------+------+-----------+-------+------------------+---------+
|       NULL|      name|  city|      state|country| registration_date|     NULL|
|          0|Customer_0|  Pune|Maharashtra|  India|        2023-01-19|     true|
|          1|Customer_1|  Pune|West Bengal|  India|        2023-08-10|     true|
|          2|Customer_2| Delhi|Maharashtra|  India|        2023-08-05|     true|
|          3|Customer_3|Mumbai|  Telangana|  India|        2023-06-04|     true|
+-----------+----------+------+-----------+-------+------------------+---------+



In [16]:
spark.sql('Select count(*) from customers_500mb').show()

+--------+
|count(1)|
+--------+
| 5286939|
+--------+



### Caching in spark tables, caching is eager , as we hit caching command it would start to cache data, which could be seen in storage.

In [17]:
spark.sql('CACHE TABLE default.customers_500mb')

DataFrame[]

In [19]:
spark.sql('SELECT * FROM customers_500mb limit 10').show()

+-----------+----------+-------+-----------+-------+------------------+---------+
|customer_id|      name|   city|      state|country|registartaion_date|is_active|
+-----------+----------+-------+-----------+-------+------------------+---------+
|       NULL|      name|   city|      state|country| registration_date|     NULL|
|          0|Customer_0|   Pune|Maharashtra|  India|        2023-01-19|     true|
|          1|Customer_1|   Pune|West Bengal|  India|        2023-08-10|     true|
|          2|Customer_2|  Delhi|Maharashtra|  India|        2023-08-05|     true|
|          3|Customer_3| Mumbai|  Telangana|  India|        2023-06-04|     true|
|          4|Customer_4|  Delhi|  Karnataka|  India|        2023-03-15|    false|
|          5|Customer_5|Kolkata|West Bengal|  India|        2023-08-19|     true|
|          6|Customer_6|Kolkata| Tamil Nadu|  India|        2023-04-21|    false|
|          7|Customer_7| Mumbai|  Telangana|  India|        2023-05-23|     true|
|          8|Cus

In [20]:
spark.sql('Select count(*) from customers_500mb').show()

+--------+
|count(1)|
+--------+
| 5286939|
+--------+



In [21]:
spark.sql('SELECT city , count(*) from customers_500mb group by city').show()

+---------+--------+
|     city|count(1)|
+---------+--------+
|    Delhi|  661025|
|  Kolkata|  660174|
|Hyderabad|  662281|
|     city|       1|
|Bangalore|  661013|
|Ahmedabad|  660218|
|  Chennai|  660249|
|   Mumbai|  661241|
|     Pune|  660737|
+---------+--------+



#### These are much faster as the data is being read from cache , insted of going through tables.
## Uncaching the table

In [22]:
spark.sql('UNCACHE TABLE customers_500mb')

DataFrame[]

In [23]:
# Lazy cahing / uncahing
spark.sql('CACHE LAZY TABLE customers_500mb')

DataFrame[]

In [24]:
spark.sql('SELECT city , count(*) from customers_500mb group by city').show()

+---------+--------+
|     city|count(1)|
+---------+--------+
|    Delhi|  661025|
|  Kolkata|  660174|
|Hyderabad|  662281|
|     city|       1|
|Bangalore|  661013|
|Ahmedabad|  660218|
|  Chennai|  660249|
|   Mumbai|  661241|
|     Pune|  660737|
+---------+--------+



In [26]:
spark.sql('show tables').show()

+---------+---------------+-----------+
|namespace|      tableName|isTemporary|
+---------+---------------+-----------+
|  default|customers_500mb|      false|
+---------+---------------+-----------+



## All cache removal , no cached data

In [28]:
spark.catalog.clearCache()